In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import sparse
import inference
import cluster
import generation

%load_ext autoreload
%autoreload 2

In [ ]:
# Matplotlib settings

basic_line_color = '#808080'
black = '#222222'
darkgray = '#A9A9A9'
highlight = '#00B2EE'

color0 = '#1f78b4'
color1 = '#b2df8a'
color2 = '#a6cee3'

plt.rc('figure', figsize=(8, 5))
plt.rc('font', size=16)
plt.rc('font', family='sans-serif')
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['lines.linewidth'] = 4
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.edgecolor'] = basic_line_color
plt.rcParams['xtick.color'] = basic_line_color
plt.rcParams['ytick.color'] = basic_line_color
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['lines.markersize'] = 12

%config InlineBackend.figure_format = 'retina'

In [ ]:
n = 3**8
snr = 7
cbar = 40
n_levels = 1
groups_per_level = 27

graph = generation.create2paramGHRG(n,snr,cbar,n_levels,groups_per_level)
A = graph.sample_network()

In [ ]:
hier = inference.infer_hierarchy(A)

In [ ]:
Ks = np.arange(1,28)
partition = hier[0]
Eagg, Nagg = partition.count_links_between_groups(A)
Aagg = Eagg / Nagg
partition_list, all_errors = inference.identify_next_level(Aagg, Ks, reps=1000)

In [ ]:
fig = plt.figure(figsize=(21, 5))
widths = [5, 8, 8]
heights = [5]
spec5 = fig.add_gridspec(ncols=3, nrows=1, width_ratios=widths,
                           height_ratios=heights)
row = 0
col=0
ax = fig.add_subplot(spec5[row, col])

plt.spy(A, ms=.01, rasterized=True, color=color0)
plt.xticks([])
plt.yticks([])
plt.xlabel('Adjacency matrix')
ax.annotate('A', (-0.11, 1), xycoords='axes fraction', va='center', size=26, weight='bold')

col=1
ax = fig.add_subplot(spec5[row, col])
# plot expected error
errs = []
err0 = inference.expected_errors_random_projection([1,27])
x = np.arange(1,28)
plt.plot(x, err0, ':', color=color2)
plt.plot(x,np.max(all_errors,axis=1))

plt.xlabel(r'Number of groups $k$')
plt.ylabel(r'Projection error $\epsilon$')
plt.legend([r'$\epsilon_0(k)$',r'$\max\;\epsilon(k)$'], frameon=False)
ax.annotate('B', (-0.11, 1), xycoords='axes fraction', va='center', size=26, weight='bold')



col=2
ax = fig.add_subplot(spec5[row, col])

norm_errors = np.zeros_like(all_errors)
for i in range(np.shape(norm_errors)[1]):
    norm_errors[:,i] = all_errors[:,i]/err0
norm_errors[0] = 1
norm_errors[-1] = 1

k1 = 2
k2 = 8
k3 = 14

max_error0 = np.max(norm_errors[k1])
max_error1 = np.max(norm_errors[k2])
max_error2 = np.max(norm_errors[k3])

plt.hist(norm_errors[k1], color=color0, label=f'$k={Ks[k1]}$')
plt.hist(norm_errors[k2], color=color1, label=f'$k={Ks[k2]}$')
plt.hist(norm_errors[k3], color=color2, label=f'$k={Ks[k3]}$')

plt.axvline(max_error0, ls=':', color=color0, lw=3)
plt.axvline(max_error1, ls=':', color=color1, lw=3)
plt.axvline(max_error2, ls=':', color=color2, lw=3)

plt.plot([norm_errors[k1][-1],norm_errors[k1][-1]], [0,200], ls='--', color=color0, lw=3)
plt.plot([norm_errors[k2][-1],norm_errors[k2][-1]], [0,200], ls='--', color=color1, lw=3)
plt.plot([norm_errors[k3][-1],norm_errors[k3][-1]], [0,200], ls='--', color=color2, lw=3)

plt.axvline(0.2, color='k', lw=2, label='significance threshold')
plt.xlim(0,max_error1+0.1)
plt.xlabel('Relative projection error')
plt.ylabel('frequency')
plt.legend(frameon=False, loc=9)
ax.annotate('C', (-0.12, 1), xycoords='axes fraction', va='center', size=26, weight='bold')

fig.tight_layout()
plt.savefig('model_select_degenerate.pdf')